# 00 — Bootstrap on Start (RunPod / fresh machine)

This notebook is the **quick-start checklist** to rehydrate the environment after a restart.
It’s designed to be safe to re-run. Adjust `PROJECT_DIR` below if your repo lives elsewhere.

In [ ]:

# Set your project root (adjust if needed)
PROJECT_DIR = "/workspace/jointdit"
print("PROJECT_DIR =", PROJECT_DIR)



## 1) (Optional) Create/activate virtualenv and install deps

Skip if your `.venv` already exists. If you run these in a raw Jupyter kernel, you may
prefer executing them in a terminal instead.


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"

# Create venv if missing
if [ ! -d .venv ]; then
  python3 -m venv .venv
fi

# Activate & install (idempotent)
source .venv/bin/activate
python --version
pip install --upgrade pip
# If your project uses pyproject.toml / editable installs:
# pip install -e .



## 2) Basic system sanity


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"

# Activate env for the rest of this cell
source .venv/bin/activate || true

echo "Python:" $(python --version)
echo "Pip:" $(pip --version)

echo "CUDA / GPU (ok if this fails locally):"
nvidia-smi || echo "No GPU visible."

echo "Torch CUDA build:"
python - <<'PY'
try:
    import torch
    print("torch", torch.__version__, "cuda?", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))
except Exception as e:
    print("Torch import error:", e)
PY



## 3) Recommended env vars for PyTorch memory


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"
source .venv/bin/activate

export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:true,max_split_size_mb:128
echo "PYTORCH_CUDA_ALLOC_CONF=$PYTORCH_CUDA_ALLOC_CONF"



## 4) Quick model/layout checks
These mirror what we validated during smoke tests.


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"
source .venv/bin/activate

# Show the smoke scripts where we expect them
ls -al scripts/smoke || true
grep -nE 'svd_smoke\.py|aldm2_smoke\.py' Makefile || true


In [ ]:

# Verify UNet cross_attention_dim == 1024 (if config exists)
import json
import pathlib

cfg_path = pathlib.Path(f"{PROJECT_DIR}/assets/models/audioldm2/unet/config.json")
if cfg_path.exists():
    cfg = json.loads(cfg_path.read_text())
    print("Found:", cfg_path)
    print("cross_attention_dim:", cfg.get("cross_attention_dim"))
else:
    print("No UNet config found at", cfg_path)



## 5) Smoke tests
Runs both SVD and ALDM2 quick checks.


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"
source .venv/bin/activate

# If Makefile contains smoke targets:
if grep -q "smoke-aldm2" Makefile && grep -q "smoke-svd" Makefile; then
  make smoke-svd
  make smoke-aldm2
else
  # Fallback: call Python directly
  python scripts/smoke/svd_smoke.py
  python scripts/smoke/aldm2_smoke.py
fi
